## 1. Data Preparation

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from PIL import ImageFile

In [3]:
# pip install torch torchvision matplotlib pandas scikit-learn

In [4]:
# Define directory paths
train_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\train"
val_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\val"
test_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\test"

In [5]:
# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [6]:
# Define transformations for training and validation datasets - Pipelines
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.RandomHorizontalFlip(),  # Randomly flip images for augmentation
        transforms.ToTensor(),  # Convert PIL images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.ToTensor(),  # Convert PIL images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ]),
}

In [7]:
BATCH_SIZE = 32

In [8]:
# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['val'])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Display the sizes of the datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 9246
Number of validation samples: 829
Number of test samples: 506


In [9]:
# Check classes in the training dataset
classes = os.listdir(train_dir)

In [10]:
classes

['fractured', 'not fractured']

## 2. Build the CNN Model

In [11]:
# Define the Custom CNN architecture
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # RGB input
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 256)  # Adjust based on image size after pooling
        self.fc2 = nn.Linear(256, 1)  # Binary classification output

    def forward(self, x):
        # Forward pass through convolutional layers and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor
        x = F.relu(self.fc1(x))  # Activation for first fully connected layer
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

In [12]:
# Set device for GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model and move it to the specified device
model = CustomCNN().to(device)

In [13]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # For binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Learning rate can be tuned

## 3. Train the Model

In [14]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# Instantiate the model, define loss function and optimizer
model = CustomCNN().to(device)
criterion = nn.BCELoss()  # For binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.view(-1), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.float().to(device)
                outputs = model(images)
                loss = criterion(outputs.view(-1), labels)
                val_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

In [17]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

c:\Users\daans\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Train Loss: 0.5222, Val Loss: 0.3603


KeyboardInterrupt: 

In [1]:
import seaborn as sns

# Evaluate the model on the test set
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = (outputs.view(-1) > 0.5).float()  # Convert probabilities to binary predictions
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Print classification report
    print(classification_report(all_labels, all_preds, target_names=classes))

    # Compute and plot confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.show()

# Assuming you have your test_loader already defined
evaluate_model(model, test_loader)

NameError: name 'model' is not defined